### Set up

#### 1. Set  up  accounts and role

In [1]:
#!pip install sagemaker==1.39.0

In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


#### 2. Setup image and instance type

In [3]:
pytorch_custom_image_name="ppi-extractor:gpu-1.0.0-201910130520"
instance_type = "ml.p3.8xlarge" 

In [4]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

#### 3. Configure train/ test and validation datasets

In [5]:
bucket = "aegovan-data"

In [6]:
plain_trainfile = "s3://{}/aimed/AIMedFull.json".format(bucket)
trainfile = "s3://{}/aimed/AIMedFull_preprocessed.json".format(bucket)

#trainfile="s3://{}/aimed/AIMedFull_Ylhsieh.json".format(bucket)
    
valfile="s3://{}/aimed/AIMedval_preprocessed.json".format(bucket)
#trainfile = "s3://{}/aimed/AIMedtrain_pubmedoverlap.json".format(bucket)
#valfile="s3://{}/aimed/AIMedval_pubmedoverlap.json".format(bucket)
#embeddingfile="s3://{}/embeddings/PubMed-and-PMC-w2v.bin.txt".format(bucket)
#embeddingfile="s3://{}/embeddings/bio_nlp_vec/PubMed-shuffle-win-30.bin.txt".format(bucket)
embeddingfile="s3://{}/embeddings/bio_nlp_vec/PubMed-shuffle-win-2.bin.txt".format(bucket)
embed_dim=200

#Collobert embedding
coll_embeddingfile="s3://{}/embeddings/collobert/words_vocab_collabert.txt".format(bucket)
coll_embed_dim=50
s3_output_path= "s3://{}/results/".format(bucket)
s3_code_path= "s3://{}/aimed_code".format(bucket)

### Start training

In [7]:
pub_inputs = {
    "train" : trainfile,
 #   "val" :valfile,
    "embedding" : embeddingfile
}

In [8]:
plain_pub_inputs = {
    "train" : plain_trainfile,
 #   "val" :valfile,
    "embedding" : embeddingfile
}

In [9]:
coll_inputs = {
    "train" : trainfile,
   # "val" :valfile,
    "embedding" : coll_embeddingfile
}

In [10]:
# hyperparameters = {
#     "dataset":"PpiAimedDatasetFactory",
#     "trainfile":trainfile.split("/")[-1],
#     "valfile":valfile.split("/")[-1],
#     "embeddingfile":embeddingfile.split("/")[-1],
#     "embeddim":embed_dim,
#     "batchsize": "32",
#     "epochs" : "1000",   
#     "log-level" : "INFO",
#     "lstmhiddensize": 100,
#     "fclayersize": 15,
#     "numlayers":7,
#     "poolingkernelsize":10,
#     "learningrate":.001,
#     "cnn_output":100,
#     "earlystoppingpatience":20
# }

In [11]:
choi_CnnNetworkNoPosFactoryhyperparameters = {
    "dataset":"PpiAimedDatasetPreprocessedFactory",
    "network" :"RelationExtractorCnnNetworkNoPosFactory",
    "trainfile":trainfile.split("/")[-1],
    "embeddingfile":coll_embeddingfile.split("/")[-1],
    "embeddim":coll_embed_dim,
    "batchsize": "32",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.001,
    "cnn_output":100,
    "earlystoppingpatience":20,
    "dropout_rate_cnn":.2,
    "fc_drop_out_rate":0.5,
    "train_val_vocab_merge":1
}

In [12]:
choi_CnnPosNetworkFactory = {
    "dataset":"PpiAimedDatasetPreprocessedFactory",
    "network" :"RelationExtractorCnnPosNetworkFactory",
    "trainfile":trainfile.split("/")[-1],
    "embeddingfile":coll_embeddingfile.split("/")[-1],
    "embeddim":coll_embed_dim,
    "batchsize": "32",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.001,
    "cnn_output":100,
    "earlystoppingpatience":20,
    "dropout_rate_cnn":.2,
    "fc_drop_out_rate":0.5,
      "train_val_vocab_merge":1
}

In [13]:
BilstmNetworkFactoryhyperparameters = {
    "dataset":"PpiAimedDatasetPreprocessedFactory",
    "network" :"RelationExtractorBiLstmNetworkFactory",
    "trainfile":trainfile.split("/")[-1],
    "valfile":valfile.split("/")[-1],
    "embeddingfile":embeddingfile.split("/")[-1],
    "embeddim":embed_dim,
    "batchsize": "50",
    "epochs" : "1000",  
    "earlystoppingpatience":20,
    "log-level" : "INFO",
    "learningrate":.001,
    "lstm_dropout":0.5,
    "pooling_kernel_size":3,
    "lstm_num_layers" :3,
    "lstm_hidden_size":64,
    "fc_layer_size":64,
    "fc_drop_out_rate":0.5,
}

In [14]:
base_experiment_bilstm_pos =  {
    "dataset":"PpiAimedDatasetFactory",
    "network" :"RelationExtractorDynamicEntityBiLstmNetworkFactory",
    "trainfile":plain_trainfile.split("/")[-1],
    "embeddingfile":embeddingfile.split("/")[-1],
    "embeddim":embed_dim,
    "batchsize": "50",
    "epochs" : "1000",  
    "earlystoppingpatience":20,
    "log-level" : "INFO",
    "learningrate":.001,
    "lstm_dropout":0.5,
    "pooling_kernel_size":3,
    "lstm_num_layers" :3,
    "lstm_hidden_size":64,
    "fc_layer_size":64,
    "fc_drop_out_rate":0.5,
    "train_val_vocab_merge":0,
    "use_min_dict":0,
    "fine_tune_embeddings":0
    
}

#### For BiLSTM network f - score using the following HP

```bash
/usr/bin/python -m main_train_k_fold --batchsize 64 --dataset PpiAimedDatasetPreprocessedFactory --earlystoppingpatience 20 --embeddim 200 --embeddingfile PubMed-shuffle-win-2.bin.txt --epochs 1000 --fc_drop_out_rate 0.5 --learningrate 0.001 --log-level INFO --lstm_dropout 0.5 --lstm_hidden_size 400 --lstm_num_layers 1 --network RelationExtractorBiLstmNetworkFactoryNoPos --train_val_vocab_merge 1 --trainfile AIMedFull_preprocessed.json
```

**precision, recall, fscore, support** for 10 folds

```json
[(0.7528089887640449, 0.7528089887640449, 0.752808988764045, None),
 (0.6915887850467289, 0.7628865979381443, 0.7254901960784315, None),
 (0.7586206896551724, 0.6226415094339622, 0.6839378238341969, None),
 (0.7727272727272727, 0.6296296296296297, 0.6938775510204083, None),
 (0.75, 0.6990291262135923, 0.7236180904522612, None),
 (0.7525773195876289, 0.7525773195876289, 0.752577319587629, None),
 (0.6851851851851852, 0.74, 0.7115384615384615, None),
 (0.7142857142857143, 0.7476635514018691, 0.730593607305936, None),
 (0.7560975609756098, 0.6326530612244898, 0.6888888888888889, None),
 (0.6788990825688074, 0.7789473684210526, 0.7254901960784315, None)]
```

**tn, fp, fn, tp** for 10 folds


```json
[(473, 22, 22, 67), 
 (454, 33, 23, 74), 
 (457, 21, 40, 66), 
 (456, 20, 40, 68), 
 (456, 24, 31, 72), 
 (462, 24, 24, 73),
 (449, 34, 26, 74), 
 (444, 32, 27, 80), 
 (465, 20, 36, 62),
 (453, 35, 21, 74)]
```


In [15]:
BilstmNetworkFactoryhyperparametersNoPos = {
    "dataset":"PpiAimedDatasetPreprocessedFactory",
    "network" :"RelationExtractorBiLstmNetworkFactoryNoPos",
    "trainfile":trainfile.split("/")[-1],
   # "valfile":valfile.split("/")[-1],
    "embeddingfile":embeddingfile.split("/")[-1],
    "embeddim":embed_dim,
    "batchsize": "64",
    "epochs" : "1000",  
    "earlystoppingpatience":20,
    "log-level" : "INFO",
    "learningrate":.001,
    "lstm_dropout":0.5,
    "lstm_num_layers" :1,
    "lstm_hidden_size":400,
    "fc_drop_out_rate":0.5,
      "train_val_vocab_merge":1
}

In [16]:


PpiAimedDatasetFactoryYlhsiehBiLstmNetwork = {
    "dataset":"PpiAimedDatasetFactoryYlhsieh",
    "network" :"RelationExtractorBiLstmNetworkFactoryNoPos",
    "trainfile":trainfile.split("/")[-1],
   # "valfile":valfile.split("/")[-1],
    "embeddingfile":embeddingfile.split("/")[-1],
    "embeddim":embed_dim,
    "batchsize": "16",
    "epochs" : "1000",  
    "earlystoppingpatience":20,
    "log-level" : "INFO",
    "learningrate":.001,
    "lstm_dropout":0.5,
    "lstm_num_layers" :1,
    "lstm_hidden_size":400,
    "fc_drop_out_rate":0.5,
  "train_val_vocab_merge":1
}

In [17]:
ResnetCnnPosNetworkFactoryhyperparameters = {
    "dataset":"PpiAimedDatasetPreprocessedFactory",
    "network" :"RelationExtractorResnetCnnPosNetworkFactory",
    "earlystoppingpatience" : 20,
    "trainfile":trainfile.split("/")[-1],
   # "valfile":valfile.split("/")[-1],
    "embeddingfile":embeddingfile.split("/")[-1],
    "embeddim":embed_dim,
    "batchsize": "8",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "dropout_rate_cnn": 0.5,
    "pooling_kernel_size": 3,
    "pool_stride":2,
    "cnn_kernel_size":3,
    "cnn_num_layers":3,
    "cnn_output":64,
    "learningrate":.00001,
    "weight_decay":.00001,
    "fc_layer_size": 256,
    "fc_drop_out_rate": 0.5,
    "input_drop_out_rate" : 0.2
   
}

In [18]:
SimpleResnetCnnPosNetworkFactoryhyperparameters = {
    "dataset":"PpiAimedDatasetFactory",
    "network" :"RelationExtractorSimpleResnetCnnPosNetworkFactory",
    "earlystoppingpatience" : 50,
    "trainfile":trainfile.split("/")[-1],
  #  "valfile":valfile.split("/")[-1],
    "embeddingfile":embeddingfile.split("/")[-1],
    "embeddim":embed_dim,
    "batchsize": "32",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "dropout_rate_cnn": 0.5,
    "pooling_kernel_size": 3,
    "pool_stride":2,
    "cnn_kernel_size":3,
    "cnn_num_layers":2,
    "cnn_output":64,
    "learningrate":.001,
    "weight_decay":.00001,
    "fc_layer_size": 256,
    "fc_drop_out_rate": 0.5,
    "input_drop_out_rate" : 0.2,
   
    "train_val_vocab_merge":1
}

In [19]:
SimpleResnetCnnPosNetworkFactoryhyperparameters_coll = {
    "dataset":"PpiAimedDatasetFactory",
    "network" :"RelationExtractorSimpleResnetCnnPosNetworkFactory",
    "earlystoppingpatience" : 50,
    "trainfile":trainfile.split("/")[-1],
  #  "valfile":valfile.split("/")[-1],
    "embeddingfile":coll_embeddingfile.split("/")[-1],
    "embeddim":50,
    "batchsize": "32",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "dropout_rate_cnn": 0.5,
    "pooling_kernel_size": 3,
    "pool_stride":2,
    "cnn_kernel_size":3,
    "cnn_num_layers":2,
    "cnn_output":64,
    "learningrate":.001,
    "weight_decay":.00001,
    "fc_layer_size": 256,
    "fc_drop_out_rate": 0.5,
    "input_drop_out_rate" : 0.2,
   
    "train_val_vocab_merge":1
}

In [20]:
SimpleResnetCnnPosNetworkFactoryhyperparametersv2 = {
    "dataset":"PpiAimedDatasetFactory",
    "network" :"RelationExtractorSimpleResnetCnnPosNetworkFactory",
    "earlystoppingpatience" : 50,
    "trainfile":trainfile.split("/")[-1],
    "valfile":valfile.split("/")[-1],
    "embeddingfile":embeddingfile.split("/")[-1],
    "embeddim":embed_dim,
    "batchsize": "8",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "dropout_rate_cnn": 0.5,
    "pooling_kernel_size": 3,
    "pool_stride":2,
    "cnn_kernel_size":3,
    "cnn_num_layers":3,
    "cnn_output":32,
    "learningrate":.001,
    "weight_decay":.00001,
    "fc_layer_size": 128,
    "fc_drop_out_rate": 0.5,
    "input_drop_out_rate" : 0.2,
    "train_val_vocab_merge":1
   
}

In [21]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainFScore",
                     "Regex": "###score: train_fscore### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationFScore",
                     "Regex": "###score: val_fscore### (\d*[.]?\d*)"}
                    ]

In [22]:
!git log -1 | head -1
!git log -1 | head -5 | tail -1

commit 08cc232d34fd2b7a68bae0387f4fc1f61c87cd15
    Add sample dataset


In [23]:
git_config = {'repo': 'https://github.com/elangovana/PPI-typed-relation-extractor.git',
              'branch': 'master',
            #  'commit': '58a09e154935248667062a36fdae7d86b86b477c'
             }

In [24]:
base_job_name="aimed-base-bilstm-pos"
hyperparameters = base_experiment_bilstm_pos #BilstmNetworkFactoryhyperparametersNoPos #PpiAimedDatasetFactoryYlhsiehBiLstmNetwork #SimpleResnetCnnPosNetworkFactoryhyperparameters
inputs = plain_pub_inputs 


In [25]:
hyperparameters

{'batchsize': '50',
 'dataset': 'PpiAimedDatasetFactory',
 'earlystoppingpatience': 20,
 'embeddim': 200,
 'embeddingfile': 'PubMed-shuffle-win-2.bin.txt',
 'epochs': '1000',
 'fc_drop_out_rate': 0.5,
 'fc_layer_size': 64,
 'fine_tune_embeddings': 0,
 'learningrate': 0.001,
 'log-level': 'INFO',
 'lstm_dropout': 0.5,
 'lstm_hidden_size': 64,
 'lstm_num_layers': 3,
 'network': 'RelationExtractorDynamicEntityBiLstmNetworkFactory',
 'pooling_kernel_size': 3,
 'train_val_vocab_merge': 0,
 'trainfile': 'AIMedFull.json',
 'use_min_dict': 0}

In [26]:
inputs

{'embedding': 's3://aegovan-data/embeddings/bio_nlp_vec/PubMed-shuffle-win-2.bin.txt',
 'train': 's3://aegovan-data/aimed/AIMedFull.json'}

In [27]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
     entry_point='main_train_k_fold.py',
   # entry_point='main_train.py',
                    source_dir = 'source/algorithms',
                    dependencies =['source/algorithms', 'source/datasets', 'source/preprocessor', 'source/modelnetworks'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                    git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    #train_use_spot_instances = True
                    train_volume_size=30,
                    code_location=s3_code_path,
                    train_max_run = 60 * 60 * 24 * 4,
                    base_job_name = base_job_name)

In [ ]:
estimator.fit(inputs)

2019-11-02 06:47:01 Starting - Starting the training job...
2019-11-02 06:47:02 Starting - Launching requested ML instances...
2019-11-02 06:47:59 Starting - Preparing the instances for training..

### HPO

In [ ]:
objective_metric_name ="ValidationFScore"

In [ ]:
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, CategoricalParameter, ContinuousParameter
hyperparameter_ranges = {'lstmhiddensize': IntegerParameter(40,200), #ContinuousParameter(0.01, 0.2),
                          "fclayersize": IntegerParameter(10,50),
                            "numlayers":IntegerParameter(1,10),
                        "poolingkernelsize":IntegerParameter(2,10)}

In [ ]:
hyperparameters={ "trainfile":trainfile.split("/")[-1],
    "valfile":valfile.split("/")[-1],
    "embeddingfile":embeddingfile.split("/")[-1],
     "embeddim":"200",
    "epochs": 100,
                 "earlystoppingpatience": 20,
                        "dataset":"PpiAimedDatasetFactory"}

In [ ]:

estimator = PyTorch(
     entry_point='main_train.py',
                    source_dir = 'source/algorithms',
                    dependencies =['source/algorithms', 'source/datasets','source/preprocessor'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                   # git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    code_location=s3_code_path,
                    #train_use_spot_instances = True
                    train_volume_size=30,
                    base_job_name ="aimed-ppi-extractor")

In [ ]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=50,
                            max_parallel_jobs=7,
                            strategy="Random",
                            base_tuning_job_name="hpo-aimed-ppi-extractor")
tuner.fit(inputs)